In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import cvxpy as cp
import plotly.graph_objects as go
from tqdm.auto import tqdm
import statsmodels.api as sm
import warnings
from datetime import datetime, timedelta
from joblib import Parallel, delayed

# print(plt.style.available) #list of available styles
#plt.style.use('ggplot')
# Configura el estilo de Seaborn para que los gráficos se vean más atractivos
sns.set(style="whitegrid")

plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 100
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
precios_sp500 = pd.read_csv("../data/sp500_precios.csv", index_col=0, parse_dates=True)
precios_sp500.head(5)

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS,SPY
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,63.381710,31.963160,37.893341,70.646538,63.908173,26.190001,131.077469,224.570007,78.138260,35.836353,...,41.850040,53.962513,35.926987,62.568439,83.576164,95.619514,156.240005,35.226196,81.659958,230.557449
2019-01-03,61.046745,29.581663,34.118874,68.318840,60.892082,25.780001,126.602242,215.699997,73.418312,35.678822,...,41.685143,53.133984,35.945953,60.423553,81.473938,93.889969,146.880005,35.031338,78.837448,225.055740
2019-01-04,63.159786,31.530161,35.575378,70.519852,62.630009,26.389999,131.524994,226.190002,75.200783,36.501442,...,42.093063,55.093021,36.988953,62.972939,83.594421,97.096619,152.970001,36.107277,81.930641,232.594147
2019-01-07,64.500923,32.425674,35.496204,71.549126,63.567928,26.330000,131.981812,229.259995,75.673691,36.685211,...,41.910801,55.379513,37.652691,62.041607,83.503029,97.134010,155.289993,36.251289,82.423607,234.428024
2019-01-08,65.446503,31.904114,36.172874,71.881653,62.804718,26.430000,135.319580,232.679993,77.519836,37.367813,...,42.396820,55.782177,37.396687,62.624863,83.338516,94.955734,156.330002,36.581703,83.651207,236.630554


In [3]:
# Calcular rendimientos logarítmicos
rendimientos_sp500 = np.log(precios_sp500).diff().dropna()

num_act = len(rendimientos_sp500.columns)

In [4]:
#Calculamos la matriz de covarianzas y los retornos esperados
matriz_cov = rendimientos_sp500.cov().to_numpy() # Covariance matrix
retornos_esperados = rendimientos_sp500.mean().to_numpy() # Column vector expected return

In [8]:
#Variables de decisión
pesos = cp.Variable(num_act)

#Restricciones
constraints = [pesos >= 0,  # No shorting
               cp.sum(pesos) == 1, # Fully invested
               ]

#Función Objetivo
riesgo = cp.quad_form(pesos, matriz_cov) # Riesgo de la cartera
objective = cp.Minimize(riesgo) # Minimizar la varianza

ret = retornos_esperados.T @ pesos # Retorno esperado de la cartera

#Problema y resuelvo
prob = cp.Problem(objective, constraints)
resultado = prob.solve()

#Guardamos los valores de la rentabilidad y riesgo de la cartera de mínimo riesgo
min_riesgo = np.array([riesgo.value, ret.value])

In [12]:
num_riegos = 500
riesgos_lst = np.linspace(min_riesgo[0], np.diag(matriz_cov).max(), num=num_riegos)

In [10]:
def optimizar_portafolio(risk, retornos_esperados, matriz_cov, num_act):
    # Variables de decisión
    pesos = cp.Variable(num_act)

    # Función objetivo
    rentabilidad = cp.Maximize(retornos_esperados.T @ pesos)

    # Restricciones
    riesgo = cp.quad_form(pesos, matriz_cov)
    constraints = [pesos >= 0, cp.sum(pesos) == 1, riesgo <= risk]

    # Resolver problema
    prob = cp.Problem(rentabilidad, constraints)
    resultado = prob.solve(solver=cp.GUROBI)
    
    return rentabilidad.value, pesos.value

In [11]:
# Ejecución en paralelo
resultados = Parallel(n_jobs=-1)(delayed(optimizar_portafolio)(risk, retornos_esperados, matriz_cov, num_act) for risk in tqdm(riesgos_lst))

# Separar los resultados
retornos_lst, pesos_lst = zip(*resultados)

  0%|          | 0/500 [00:00<?, ?it/s]

Set parameter WLSAccessID
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2457827

In [18]:
len(pesos_lst[0])

490

In [24]:
# Elegir una paleta de colores y generar colores
from matplotlib import cm
paleta = cm.get_cmap('tab20c', num_act)
colores = [paleta(i) for i in range(num_act)]

# Crear una gráfica de áreas apiladas
tiempo = np.arange(num_riegos)  # Eje x - tiempo

plt.stackplot(tiempo, pesos_res.T, colors=colores, labels=[f'{assets[i]}' for i in range(num_act)])

# Añadir bordes a las áreas
for i in range(num_act):
    plt.plot(tiempo, np.sum(pesos_res[:, :i+1], axis=1), color='black', linewidth=0.5)

# Invertir el eje x
plt.gca().invert_xaxis()

# Añadir texto personalizado en el eje x
plt.text(num_riegos-1, -0.05, 'Mínimo riesgo', ha='center')  # Cerca del 100
plt.text(0, -0.05, 'Máximo riesgo', ha='center')         # Cerca del 0


plt.xlabel('Aversión al Riesgo: $\gamma$. Valores más altos de $\gamma$ implican menor tolerancia al riesgo')
plt.ylabel('Peso del Activo')
plt.title('Evolución de la Cartera')

# Ajustar la leyenda fuera del área del gráfico
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

plt.show();